#Conectarse a Drive
Leo los csv con pandas y creo el .db con sqlite3


In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
import csv
import sqlite3

drive.mount('/content/drive')

filenamePagos = '/content/drive/My Drive/MOPC_Test/PAGOS_UNIFICADOS.csv'
filenamePeriodos = '/content/drive/My Drive/MOPC_Test/PERIODOS_TABLEROS.csv'

#leo los csv
dfPagos = pd.read_csv(filenamePagos)
dfPeriodos = pd.read_csv(filenamePeriodos)

dfPeriodos = dfPeriodos[dfPeriodos.columns[2:]] #quito el inidices que no utilizaré

#creo .db y tablas Pago y Periodo
conn = sqlite3.connect('base.db')
c = conn.cursor()
conn.commit()

basePago = dfPagos.to_sql("Pago", conn, if_exists='append', index=False)
basePeriodo = dfPeriodos.to_sql("Periodo", conn, if_exists='append', index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Queries en SQL
Me piden una vizualizacion que muestre la evolucion temporal (de los ultimos 6 periodos) de los pagos para cada cliente

Por lo que quiero sumar los montos de los pagos agrupandondolos por cliente y los ultimos 6 periodos

Pasos:
1. ultSeis <- filtrar en Periodo y obtener los ultimos 6 periodos solicitados
2. TODO <- ultSeis  innerJoin  Pago ON IDCLIENTE  where fechaSiguiendoCondicionEstéDentroDePeriodo
3. sumar monto FROM TODO agrupando por cliente y Periodo

In [ ]:
#Asumo que los periodos (en fechas) para cada AMEX y HSBC son los mismos, y por ello modfico los IDCLIENTE
tbPeriodoEditNamesCtes = '''
SELECT DISTINCT
      CASE
        WHEN IDCLIENTE LIKE 'AMEX_%' THEN 'AMEX'
        WHEN IDCLIENTE LIKE 'HSBC_%' THEN 'HSBC'
        WHEN IDCLIENTE LIKE 'BSR_%' THEN 'BSR'
        ELSE IDCLIENTE
      END AS Cliente,
      MES, AÑO, FINICIO, FFIN
FROM Periodo
'''
tbPagoEditNamesCtes = '''
SELECT *,
      CASE
        WHEN IDCLIENTE LIKE 'AMEX_%' THEN 'AMEX'
        WHEN IDCLIENTE LIKE 'HSBC_%' THEN 'HSBC'
        WHEN IDCLIENTE LIKE 'BSR_%' THEN 'BSR'
        ELSE IDCLIENTE
      END AS Cliente
      FROM Pago
'''

#Nota al corrector: como máxima fecha de pago era el 27-07-2021 y el periodo agosto2021 comienza como mínimo el 30-07-2021, es decir: no hay pagos en agosto2021
# decido seleccionar los ultimos 7 periodos para asi obtener los ultimos 6 periodos donde SI hay pagos.
tbUlt6Periodos = '''
SELECT *
FROM (
  SELECT *, ROW_NUMBER() OVER (PARTITION BY Cliente ORDER BY AÑO DESC, MES DESC) row
  FROM (''' + tbPeriodoEditNamesCtes + ''')
  ORDER BY Cliente, row
  ) t
WHERE t.row <= 7 ORDER BY Cliente
'''

#Hago inner join con los atributos que me interesan y viendo que cumplan condicion de fecha de pago
#(tanto para los de fuente 'MEMO_PAGO_OPERADOR' como el resto)
cond = '''FUENTE_PAGO = 'MEMO_PAGO_OPERADOR' AND FECHA_REMESA IS NOT NULL '''
queryParaSuma = '''
SELECT pago.IDMOROSO, pago.FUENTE_PAGO,	pago.ID_PAGO, pago.FECHA_PAGO,	pago.FECHA_REMESA, pago.MONTO, pago.Cliente, per.MES, per.AÑO, per.FINICIO, per.FFIN
FROM (''' + tbPagoEditNamesCtes + ''' ) pago INNER JOIN (''' + tbUlt6Periodos + ''' ) per ON per.Cliente = pago.Cliente
WHERE (('''+cond+''' AND ( FECHA_REMESA  BETWEEN FINICIO AND FFIN)) OR (NOT ('''+cond+''') AND FECHA_PAGO BETWEEN FINICIO AND FFIN ))
GROUP BY ID_PAGO
'''

# Achico los atributos post hacer y para quedarme solo los que quiero mostrar en LookerStudio
queryParaVisualizar = '''SELECT Cliente, MES, AÑO, SUM(MONTO) AS SUMA_DE_PAGOS FROM  (''' + queryParaSuma + ''' ) GROUP BY Cliente, MES, AÑO '''

#EXTRA: Decido hacacer ambas visualizaciones: la que muestra todas las categorias, y la que muestra unicamente 3 como "AMEX", "HSBC" y "OTROS"
queryVisualizar3Categorias = '''
SELECT TresCtes,
MES, AÑO, SUM(SUMA_DE_PAGOS) AS SUMA_DE_PAGOS
FROM  (SELECT CASE
        WHEN Cliente = 'AMEX' THEN Cliente
        WHEN Cliente = 'HSBC' THEN Cliente
        ELSE 'OTROS'
      END AS TresCtes, MES,	AÑO, SUMA_DE_PAGOS FROM (''' + queryParaVisualizar + ''' ))
GROUP BY TresCtes, MES, AÑO '''

In [ ]:
df_Visualizacion = pd.read_sql(queryParaVisualizar, conn)
df_Visualizacion

,Cliente,MES,AÑO,SUMA_DE_PAGOS
0,AMEX,2,2021,327563.16
1,AMEX,3,2021,617093.95
2,AMEX,4,2021,220581.01
3,AMEX,5,2021,563589.77
4,AMEX,6,2021,432420.09
5,AMEX,7,2021,191064.35
6,BSR,2,2021,119199.42
7,BSR,3,2021,192575.24
8,BSR,4,2021,167848.36
9,BSR,5,2021,205029.83


In [ ]:
df_Visualizar3Ctes = pd.read_sql(queryVisualizar3Categorias, conn)
df_Visualizar3Ctes.head()

,TresCtes,MES,AÑO,SUMA_DE_PAGOS
0,AMEX,2,2021,327563.16
1,AMEX,3,2021,617093.95
2,AMEX,4,2021,220581.01
3,AMEX,5,2021,563589.77
4,AMEX,6,2021,432420.09


Pasaje a un formato de string que sea reconocible como tipo Fecha en el spreadsheet y que sea más fácil ordenar en la visualización:

In [ ]:
df_Visualizacion = df_Visualizacion.rename(columns={"AÑO": "year", "MES": "month", "SUMA_DE_PAGOS": "Suma de Pagos"})
df_Visualizacion['day'] = 1
df_Visualizacion['PERIODO'] = pd.to_datetime(df_Visualizacion[['year', 'month', 'day']]).dt.strftime('%Y-%m-%d')
df_Visualizacion = df_Visualizacion[['Cliente', 'PERIODO', 'Suma de Pagos']]

In [ ]:
df_Visualizar3Ctes = df_Visualizar3Ctes.rename(columns={"AÑO": "year", "MES": "month","SUMA_DE_PAGOS": "Suma de Pagos"})
df_Visualizar3Ctes['day'] = 1
df_Visualizar3Ctes['PERIODO'] = pd.to_datetime(df_Visualizar3Ctes[['year', 'month', 'day']]).dt.strftime('%Y-%m-%d')
df_Visualizar3Ctes = df_Visualizar3Ctes[['TresCtes', 'PERIODO', 'Suma de Pagos']]

#Guardar en un *spreadsheet*

In [ ]:
#Crear credenciales para conectarme a drive
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
miMail = 'byleilany@gmail.com' #Si quieren utilizar este collab en su drive necesitan modificar este mail y crear una carpeta en su propio Drive llamada "MOPC_Test" con los archivos que enviaron por mail

In [ ]:
filenameSheet = '/content/drive/My Drive/MOPC_Test/DataParaVisualizar'
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Dsa7zHrrp20CwtQhykH48rUShUMy_GjC4LcRF4RdIXM/edit#gid=0')
worksheet = sh.get_worksheet(0)

worksheet.update([df_Visualizacion.columns.values.tolist()] + df_Visualizacion.values.tolist())

{'spreadsheetId': '1Dsa7zHrrp20CwtQhykH48rUShUMy_GjC4LcRF4RdIXM',
 'updatedRange': 'Data!A1:C25',
 'updatedRows': 25,
 'updatedColumns': 3,
 'updatedCells': 75}

In [ ]:
# worksheet2 = sh.add_worksheet(title="data3ctes", rows=100, cols=20)
worksheet2 = sh.get_worksheet(1)
worksheet2.update([df_Visualizar3Ctes.columns.values.tolist()] + df_Visualizar3Ctes.values.tolist())

{'spreadsheetId': '1Dsa7zHrrp20CwtQhykH48rUShUMy_GjC4LcRF4RdIXM',
 'updatedRange': 'data3ctes!A1:C19',
 'updatedRows': 19,
 'updatedColumns': 3,
 'updatedCells': 57}